In [ ]:
import chatlas
import dotenv

dotenv.load_dotenv()

In [ ]:
from pyhere import here

recipe_txt = here("data/recipes/text/")
txt_cheesecake = recipe_txt / "PhillyCheesesteak.md"

In [ ]:
print(txt_cheesecake)

Here's an example of the structured output we want to achieve for a single
recipe:

```json
{
  "title": "Spicy Mango Salsa Chicken",
  "description": "A flavorful and vibrant chicken dish...",
  "ingredients": [
    {
      "name": "Chicken Breast",
      "quantity": "4",
      "unit": "medium",
      "notes": "Boneless, skinless"
    },
    {
      "name": "Lime Juice",
      "quantity": "2",
      "unit": "tablespoons",
      "notes": "Fresh"
    }
  ],
  "instructions": [
    "Preheat grill to medium-high heat.",
    "In a bowl, combine ...",
    "Season chicken breasts with salt and pepper.",
    "Grill chicken breasts for 6-8 minutes per side, or until cooked through.",
    "Serve chicken topped with the spicy mango salsa."
  ]
}
```

In [ ]:
from typing import List, Optional

from pydantic import BaseModel, Field


class Ingredient(BaseModel):
    name: str = Field(..., description="Name of the ingredient")
    quantity: float = Field(
        ...,
        description="Quantity as provided (kept as string to allow ranges or fractions)",
    )
    unit: Optional[str] = Field(
        None,
        description="Unit of measure, if applicable",
    )
    notes: Optional[str] = Field(
        None,
        description="Additional notes or preparation details",
    )


class Recipe(BaseModel):
    title: str
    description: str
    ingredients: List[Ingredient]
    instructions: List[str] = Field(..., description="Step-by-step instructions")

In [ ]:
chat = chatlas.ChatOpenAI(model="gpt-4.1-nano")
recipe = chat.chat_structured(txt_cheesecake, data_model=Recipe)

`.chat_structured()` returns an instance of the provided Pydantic model, so
you can access fields directly:

In [ ]:
recipe.title

Or you can convert it to JSON with pydantic's built-in `.model_dump_json()`
method:

In [ ]:
print(recipe.model_dump_json(indent=2))